<a href="https://colab.research.google.com/github/masatomo-ty/power_flow_calc/blob/master/power_flow_case3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
潮流計算
全ノードPQ指定
"""

import numpy as np #行列のnpコード
import pandas as pd
import math
import random

V_base = 0.416 #[kV]
P_base = 1 #[kW]
Z_base = (V_base*(10**3))**2/(P_base*(10**3))

# Import Feeder Data // NodeA to NodeB  D:距離　Load:負荷(消費)　PV:発電(需要) ############
df_line = pd.read_excel("feeder_case3.xlsx")

array_NodeA = df_line['NodeA'].to_numpy()
array_NodeB = df_line['NodeB'].to_numpy()
array_D = df_line['D[m]'].to_numpy()
array_Load = df_line['Load'].to_numpy()
array_PV = df_line['PV'].to_numpy()
array_Cable_name = df_line['Cable'].to_numpy()
##########################################################

array_NodeAB = np.concatenate([array_NodeA, array_NodeB], 0)
bus_num = np.unique(array_NodeAB).size

##Cableの種類とR,Xの値を入手
df_Cable = pd.read_excel("Cable.xlsx") 
Cable_num = len(array_NodeA) #線路の数　要素数
##線路　R+jXを行列として定義########################
Cable = np.zeros((Cable_num,), dtype=np.complex128) ##母線の数は505-1本 
for i in range(0,Cable_num): ##母線の数は505-1本
    if array_Cable_name[i] == "25_SAC_XC":
        RX = df_Cable['25_SAC_XC']
    elif array_Cable_name[i] == "2c_.007":
        RX = df_Cable['2c_.007']
    elif array_Cable_name[i] == "2c_.0225":
        RX = df_Cable['2c_.0225']
    elif array_Cable_name[i] == "2c_.04":
        RX = df_Cable['2c_.04']
    elif array_Cable_name[i] == "35_SAC_XC":
        RX = df_Cable['35_SAC_XC']
    elif array_Cable_name[i] == "4_PSC":
        RX = df_Cable['4_PSC']    
    elif array_Cable_name[i] == "4c_.06":
        RX = df_Cable['4c_.06']
    elif array_Cable_name[i] == "4c_.1":
        RX = df_Cable['4c_.1']
    elif array_Cable_name[i] == "4c_.5":
        RX = df_Cable['4c_.5']
    elif array_Cable_name[i] == "4c_300_SAC":
        RX = df_Cable['4c_300_SAC']
    elif array_Cable_name[i] == "4c_70":
        RX = df_Cable['4c_70']

    Cable[i] = (RX[0]*array_D[i] + 1j*RX[1]*array_D[i])/1000
#############################################################

##インピーダンス行列Z##################################
Z = np.zeros((bus_num+1, bus_num+1), dtype=np.complex128) ####
#Z[1,1] = Cable[1]
for i in range(0,Cable_num): ####i=0~503 母線の数は504本
    A = array_NodeA[i]
    B = array_NodeB[i]
    Z[A,B] = Cable[i]

#####################################

Z = Z/Z_base #インピーダンス行列をpu値に変換

#########アドミタンス行列Y　505母線がある　504*504の行列     
Y = np.zeros((bus_num, bus_num, ), dtype=np.complex128) #535を変える
for i in range(1,bus_num+1):#1~536
    for j in range(1,bus_num+1):#1~536
        if Z[i,j] != 0:
            Y[i-1,i-1] = Y[i-1,i-1] + 1/Z[i,j] #Y11 など　駆動点
            Y[i-1,j-1] = Y[i-1,j-1] - 1/Z[i,j] #
            Y[j-1,i-1] = Y[j-1,i-1] - 1/Z[i,j] #
            Y[j-1,j-1] = Y[j-1,j-1] + 1/Z[i,j] #Y11 など　駆動点

######################################################

#電圧-------------------------------------------------
V2 = np.zeros((bus_num+1, )) #電圧指定の母線のみ追加　蓄電池の母線506~536

df_order = pd.read_excel("order.xlsx")
array_Order = df_order['Order'].to_numpy()

########################################################

V_results = [list(range(0,37))]

Real_Powers = [np.array(range(0,38))]
Transactions = [np.array(range(0,38))]

# Load, PVのデータ取得位置
# time = 14 #時刻　t = 時刻*60(分)/5(分刻み) / 120の時10:00, 144の時12:00
j = 0 #loadの数　j=0の時No.1の負荷を得る

Total_Charge = np.zeros((bus_num+1, ))

for time in range(1,289):
# for time in range(1,13):

    ##負荷の値を入手
    df_Loads = pd.read_excel("Summer_Load_Profiles.xlsx") 
    Loads = df_Loads.to_numpy()
    Loads_PQ = np.zeros((bus_num+1, ), dtype=np.complex128) #各母線の負荷によるPQ　a行=母線aのPQ消費

    for i in range(0,Cable_num):
        if array_Load[i] == 1:
            t = time
            j = array_NodeB[i]
            Loads_PQ[array_NodeB[i]] = (Loads[t-1,j]*0.95 + 1j*Loads[t-1,j]*math.sqrt(1-0.95*0.95)) 
            #Loads_PQ[array_NodeB[i]] = Loads[t-1,j]#左は時刻　jはload数

    ##PV出力の値を入手
    df_PV = pd.read_excel("Summer_PV_Profiles.xlsx") 
    PV_profile = df_PV.to_numpy()
    PV_out = np.zeros((bus_num+1, ), dtype=np.complex128) #PV出力　a行=母線aのPVの出力

    for i in range(0,Cable_num):
        if array_PV[i] == 1:
            t = time
            j = array_NodeB[i]
            PV_out[array_NodeB[i]] = PV_profile[t-1,j]*0.95 + 1j*PV_profile[t-1,j]*math.sqrt(1-0.95*0.95)

    ### 取引
    if time == 1:
        Node_Sellable_Rate = np.zeros((bus_num+1, ))
        Node_Buyable_Rate = np.zeros((bus_num+1, ))

        for i in range(0,Cable_num):
            if array_Order[i] == 1:
                Node_Sellable_Rate[i] = random.uniform(0.05, 0.95)
            elif array_Order[i] == -1:
                Node_Buyable_Rate[i] = random.uniform(0.05, 0.95)

    if time % 12 == 0:
        for i in range(0,Cable_num):
            if array_Order[i] > 0:
                Node_Sellable_Rate[i] = Node_Sellable_Rate[i] * random.uniform(0.95, 1.05)
            elif array_Order[i] < 0:
                Node_Buyable_Rate[i] = Node_Buyable_Rate[i] * random.uniform(0.95, 1.05)

    Total_Transaction_Volume = PV_out.sum()*0.3

    if (PV_out.real - Loads_PQ.real).sum() > 0:        
        Node_Sellable = (Node_Sellable_Rate / Node_Sellable_Rate.sum()) * Total_Transaction_Volume * -1
        Node_Buyable = (Node_Buyable_Rate / Node_Buyable_Rate.sum()) * Total_Transaction_Volume * -1
        Total_Charge = Total_Charge + Node_Buyable
        Node_Transaction = Node_Sellable + Node_Buyable

        Ps = (PV_out.real - Loads_PQ.real + Node_Transaction.real)/P_base
        Qs = (PV_out.imag - Loads_PQ.imag + Node_Transaction.imag)/P_base
    else:
        Node_Transaction = np.zeros((bus_num+1, ))
        if 157 < time < 230:
            for i in range(0,Cable_num):
                if array_Order[i] == -1:
                    Node_Transaction[i] = (Total_Charge[i] / (230-157)) * -1
            Ps = (PV_out.real - Loads_PQ.real + Node_Transaction.real)/P_base
            Qs = (PV_out.imag - Loads_PQ.imag + Node_Transaction.imag)/P_base
        else:
            Ps = (PV_out.real - Loads_PQ.real)/P_base
            Qs = (PV_out.imag - Loads_PQ.imag)/P_base

    Transactions.append(Node_Transaction)
    Real_Powers.append(Ps)

    c_PsQs = range((Ps[1:bus_num+1]).shape[0])
    PsQs = np.insert((Qs[1:bus_num+1]),c_PsQs,(Ps[1:bus_num+1])[c_PsQs],axis=0) #PsとQs交互の行列　2*a行
      
    PsQs = PsQs[2:2*(bus_num)] #母線1を除くPsQs ΔPQのときに使うのは基準母線以外のため 2*(a-1)行

    #######################################################
    e = np.ones((bus_num, ))
    f = np.zeros((bus_num, ), dtype=np.complex128)

    # assign base voltage for slack node
    # e[0] = 1
    if time < 14 or time > 140:
        e[0] = 1.060
    else:
        e[0] = 1.020

    V = e + f

    n = 0
    dev = np.ones(((bus_num-1)*2, ))/10

    while max(np.absolute(dev)) > 0.000001:
    #while n<1:

        S = (np.conjugate(Y) @ np.conjugate(V)) * V
        P = S.real
        Q = S.imag
        
    #Jの要素算出###################
        ab = Y @ V
        ak = ab.real
        bk = ab.imag
        
    #　教科書 6.163式
        dPkem = Y.real * V.real + Y.imag * V.imag
        dQkfm = -Y.real * V.real - Y.imag * V.imag
        dPkfm = -Y.imag * V.real + Y.real * V.imag
        dQkem = -Y.imag * V.real + Y.real * V.imag
        dVk2em = 0 
        dVk2fm = 0       
        
    #　教科書 6.164式
        dPek = ak + (np.diag(Y)).real * V.real + (np.diag(Y)).imag * V.imag
        dPfk = bk + (-np.diag(Y)).imag * V.real + (np.diag(Y)).real * V.imag
        dQek = -bk + (-np.diag(Y)).imag * V.real + (np.diag(Y)).real * V.imag
        dQfk = ak - (np.diag(Y)).real * V.real - (np.diag(Y)).imag * V.imag
        dV2ek= 2*V.real
        dV2fk= 2*V.imag
        
        
        for i in range(2,bus_num+1):
            for j in range(2,bus_num+1):
                if i == j:
                    T = np.array([[dPek[i-1], dPfk[i-1]], 
                                  [dQek[i-1], dQfk[i-1]]])
                    if j == 2:
                        J = T
                    else:
                        J = np.concatenate([J, T], 1)
                if i != j:
                    T = np.array([[dPkem[i-1,j-1], dPkfm[i-1,j-1]], 
                                  [dQkem[i-1,j-1], dQkfm[i-1,j-1]]])
                    if j == 2:
                        J = T
                    else:
                        J = np.concatenate([J, T], 1)
                        
        #JaにJを追加していく         
            if i == 2: #Bus2の場合
                Ja = J
            else: #Bus3~
                Ja = np.concatenate([Ja, J], 0)
    #---------------------------------------------------------------------------------#Jacobian 終了   
        
        m = range((P[1:(bus_num)].real).shape[0])
        PQ_cal = np.insert((Q[1:(bus_num)]),m,(P[1:(bus_num)])[m],axis=0) #P,Q算出値の値を交互に配列　(a-1)*2行

        #---------------------------------------------------------------------
        
        #ΔPQの算出　PV指定の母線はΔQ → ΔV^2-------------------------------------------
        d_PQ = PsQs - PQ_cal #
        
        
        for i in range(1, bus_num):
            if V2[i] != 0: #　PV指定母線
                d_PQ[(i-2)*2+1] = V2[i]**2 - (e[i-1]**2+(np.absolute(f[i-1]))**2) #PV指定の場合，d_Q　→　d_V^2に変える

          
        #---------------------------------------------------------------------
        
        #Δe,Δfの算出------------------------------------------------------------
        Jac = np.linalg.inv(Ja) #J^-1
        #dev = np.linalg.inv(Ja) @ d_PQ #母線2~nまでのdev (母線総数-1)*2行 P287
        dev = np.dot(np.linalg.inv(Ja), d_PQ) #上式と同じ
        dev_0 = np.array([0,0])
        dev_1 = np.concatenate([dev_0, dev], 0) #母線1の分を追加　　母線総数*2行
        
        #---------------------------------------------------------------------
        
        #電圧の再定義　---------------------------------------------------------------
        dev_e = dev_1[::2] #Δe2~e4      #dev_1をeとfに分ける
        dev_f = dev_1[1::2] #Δf2~f4
        
        e = e + dev_e
        f = f + dev_f*1j
        V = e + f
        #--------------------------------------------------------------------------------
        n = n + 1

    V_last = np.zeros((bus_num, )) 
    for ss in range(0,bus_num):
        V_last[ss] = np.sqrt(e[ss]**2+(np.absolute(f[ss]))**2)

    if time % 12 == 0:
        print(time)
    S_last = (np.conjugate(Y) @ np.conjugate(V)) * V
    P_last = S_last.real
    Q_last = S_last.imag 
    V_results.append(V_last)


T_data = np.array(Transactions)
np.savetxt('T_real_results.csv',T_data.real,delimiter=',')
np.savetxt('T_imag_results.csv',T_data.imag,delimiter=',')

P_data = np.array(Real_Powers)
np.savetxt('P_results.csv',P_data,delimiter=',')

V_data = np.array(V_results)
np.savetxt('V_results.csv',V_data,delimiter=',')


12


In [16]:
import numpy as np
import pandas as pd
import math
import random

Selling_Frequency_Rate = np.zeros((17, ))
for i in range(0,17):
    Selling_Frequency_Rate[i] = random.uniform(0.05, 0.95)

Selling_Frequency_Rate = Selling_Frequency_Rate / Selling_Frequency_Rate.sum()

print(1 % 12)

1
